# This project implemented by Elham Mahdipour
## She is a Ph.D. Candidate of computer engineering at Yazd University, Yazd, Iran.
### Please feel free and contact to me: elham.mahdipour@gmail.com/ elham.mahdipour@stu.yazd.ac.ir

# Phase 1 
## Create Dataset and Preprocessing

In [1]:
import networkx as nx
G1=nx.read_leda('biogrid samples\SCerevisiae.gw')
G1

In [2]:
G2=nx.read_edgelist('biogrid samples\HSapiens.gw')
G2

In [3]:
### Check and Swap if G1 > G2 ###
print(len(G1))
print(len(G2))
print('*******')
if len(G1)>len(G2):
    temp=G1
    G1=G2
    G2=temp
print(len(G1))
print(len(G2))

5831
13276
*******
5831
13276


In [4]:
G_target_na=nx.read_weighted_edgelist('biogrid samples\SC-HS.tsv')
print(len(G_target_na))
G_target_na  

2328


In [5]:
ed1=G1.edges()
ed2=G2.edges()

nd1=G1.nodes()
nd2=G2.nodes()

el1=list(ed1)
el2=list(ed2)

nd1=list(nd1)
nd2=list(nd2)

degG1 = [val for (node, val) in G1.degree()]
degG2 = [val for (node, val) in G2.degree()]

# Feature Extraction

# compute score for create similarity matrix

In [6]:
def deg_Diff(G1,G2):
    Degree_Difference=np.zeros((len(G1),len(G2)))
    for i in range(len(G1)):
        for j in range(len(G2)):
            Degree_Difference[i][j]=abs(degG1[i]-degG2[j])/max(degG1[i],degG2[j])
    return Degree_Difference

In [7]:
def score_pageRank(X):
    a=nx.pagerank(X)
    return a

In [8]:
def coefficient_pagerank(x,y):  #x is G1, y is G2
   # print(len(x))
    p1=score_pageRank(x)
    b=p1.values()
    pr1=list(b)
    p2=score_pageRank(y)
    c=p2.values()
    pr2=list(c)
    pr=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            #print(pr1[i],pr2[j])
            pr[i][j]=abs(pr1[i]-pr2[j])/max(pr1[i],pr2[j])   #minimum pr is maximum similarity of topology 
    return pr

In [9]:
def coefficient_edges(index_node, G, GraphNumber):
    if GraphNumber==1:
        sum_edge=0        
        for i in G.neighbors(nd1[index_node]):                       
            sum_edge=sum_edge+degG1[nd1.index(i)]
        #print(sum_edge)
        temp=(degG1[index_node]-1) if degG1[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG1[index_node]*temp)
    else:
        sum_edge=0        
        for i in G.neighbors(nd2[index_node]):                       
            sum_edge=sum_edge+degG2[nd2.index(i)]
        #print(sum_edge)
        temp=(degG2[index_node]-1) if degG2[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG2[index_node]*temp)
    return coeff_node

In [10]:
def compute_Ea(G1,G2):
    Ea_G1=np.zeros(len(G1))
    Ea_G2=np.zeros(len(G2))
    for i in range(len(G1)):
        Ea_G1[i]=coefficient_edges(i, G1, 1)
    for j in range(len(G2)):
        Ea_G2[j]=coefficient_edges(j,G2,2)
    ea=[Ea_G1, Ea_G2]
    return(ea)

In [11]:
# compute relative clustering coefficient difference between node a (in G1) and node b (in G2)
def CD(G1, G2):
    cd=np.zeros((len(G1),len(G2)))
    EA=compute_Ea(G1,G2)
    #print(EA[0])      #Ea for G1
    #print("===================")
    #print(EA[1])      #Ea for G2
    for i in range(len(G1)):
        for j in range(len(G2)):
            cd[i,j]=abs(EA[0][i]-EA[1][j])/max(EA[0][i],EA[1][j])
    return cd

In [12]:
elt=G_target_na.edges()
elt=list(elt)
print(len(elt))
elt

1164


[('850996', '60558'),
 ('854309', '93973'),
 ('850998', '8892'),
 ('855386', '3065'),
 ('855892', '9212'),
 ('852740', '9276'),
 ('852709', '27032'),
 ('852862', '201595'),
 ('853201', '6836'),
 ('856079', '38'),
 ('852066', '285195'),
 ('852594', '6576'),
 ('851342', '523'),
 ('851052', '6175'),
 ('851063', '1468'),
 ('853757', '9945'),
 ('855104', '5976'),
 ('854392', '5981'),
 ('852383', '5983'),
 ('851065', '56954'),
 ('852456', '51144'),
 ('851098', '23395'),
 ('851102', '55697'),
 ('856753', '8766'),
 ('851582', '275'),
 ('855480', '2539'),
 ('853222', '2021'),
 ('856608', '58497'),
 ('851928', '6500'),
 ('855474', '833'),
 ('852665', '11198'),
 ('855428', '9125'),
 ('853371', '9126'),
 ('856402', '54938'),
 ('856419', '23203'),
 ('856669', '957'),
 ('852762', '991'),
 ('851117', '56931'),
 ('854986', '353'),
 ('851121', '11062'),
 ('851125', '134430'),
 ('851006', '2678'),
 ('852244', '11212'),
 ('851768', '908'),
 ('856815', '9277'),
 ('855602', '9868'),
 ('856432', '8659'),
 (

In [13]:
def sequence_score(x,y):
    seq=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            q1=(nd1[i],nd2[j]) in elt
            if q1==False:
                c=0
            else:
               # print(q1)
                c=1
            seq[i][j]=c    
            
    return seq

In [14]:
def compute_score(G1,G2):
    coeff_pr=coefficient_pagerank(G1,G2)
    dd=deg_Diff(G1,G2)
    cd=CD(G1,G2)
    seq_sc=sequence_score(G1,G2)
    
    alpha=0.1
    betta=0.2
    gamma=0.2
    zetta=1-alpha-betta-gamma
    s=alpha*(1-coeff_pr)+betta*(1-dd)+gamma*(1-cd)+zetta*seq_sc
    return s,coeff_pr, dd, cd, seq_sc

In [15]:
import numpy as np
m, coeff_pr, dd, cd, seq=compute_score(G1,G2)
sim=m

# Phase 2: Dataset Generation

# create data for deep learning

# change problem to classification 
## [node of G1, node of G2, sequence similarity, Coefficient page rank, clustering coefficient difference,  similarity score, alignment=yes(0)/TN or no(1)/TP]

In [16]:
data=[]
noc=0
yc=0
for i in range(len(nd1)):
    for j in range(len(nd2)):
        if G_target_na.has_edge(nd1[i],nd2[j]):
            align_class='Yes'
            sample=[int(nd1[i]),int(nd2[j]),seq[i][j], coeff_pr[i][j], dd[i][j],cd[i][j],sim[i][j],align_class] 
            data.append(sample)
            yc+=1
        else:
            align_class='No'
            sample=[int(nd1[i]),int(nd2[j]),seq[i][j], coeff_pr[i][j], dd[i][j],cd[i][j],sim[i][j],align_class] 
            data.append(sample)
            noc+=1
print(len(data), yc, noc)

77412356 1045 77411311


In [17]:
from sklearn.utils import shuffle
data = shuffle(data)
X=[]
y=[]
for i in range(len(data)):
    X.append(data[i][0:7])
    y.append(data[i][7])
    
len(X),len(y)

(77412356, 77412356)

In [18]:
X_train=X
y_train=y

# test model for real data

In [19]:
# Label encode Class (Species)
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# One Hot Encode
y_train = np_utils.to_categorical(encoded_Y)

Using TensorFlow backend.


In [20]:
import numpy as np
X_train=np.array(X_train)
y_train=np.array(y_train)

# Phase 3: R-NA network

## Load tune R-NA model

In [ ]:
from keras.models import load_model
#del model
model_rnn = load_model('biogrid samples/deep_model_resample_RNN_SC-AT.h5')
model_rnn.load_weights('biogrid samples/deep_model_resample_RNN_weights_SC-AT.h5', by_name=True)

In [25]:
# without resample
result_tr = model_rnn.evaluate(X_train, y_train)
result_tr

77412356/77412356 [==============================] - 4410s 57us/step
[5.2389153945428462423e-6, 1.0, 7.658966205781326e-06, 7.532384915975854e-11]


In [31]:
%%time
out_tr=model_rnn.predict(X_train)
out_tr

# Create alignment file 
f = open("biogrid samples/align-SC-HS.txt", "w")
for i in range(len(out_tr)):
    if out_tr[i][1]>out_tr[i][0]:
        z=[X_train[i][0],X_train[i][1]]
        f.write(str(z))
f.close()

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train.argmax(axis=1), out_tr.argmax(axis=1)))

from sklearn.metrics import f1_score, precision_score, recall_score
print(precision_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(recall_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(f1_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))

[[77411311     0]
 [       0     1045]]
1.0
1.0
1.0
Wall time: 45min 30s


# test other classifier without resample

In [ ]:
x_tra,y_tra=X_train,y_train

In [ ]:
X_train,y_train=X,y

In [ ]:
%%time
# Linear Discriminant Analysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = lda.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Accuracy of LDA classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999865008629889
[[77411311        0]
 [    1045        0]]


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.49999325043149445
0.5


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.4999966251929687
Wall time: 23min 37s


In [ ]:
%%time
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = knn.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

Accuracy of KNN classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999865008629889
[[77411311        0]
 [    1045        0]]


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.49999325043149445
0.5


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.4999966251929687
Wall time: 2h 18min 44s


In [37]:
%%time
# SVM
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(svm.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = svm.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

Accuracy of SVM classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999874016829726
[[77411311        0]
 [    1045        0]]
0.49999426304136743
0.5
0.4999976321826531
Wall time: 4h 38min 54s
